In [79]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_val_score, train_test_split

In [80]:
data = pd.read_csv(r"D:\download\ruc_Class25Q1_train.csv")
data_test = pd.read_csv(r"D:\download\ruc_Class25Q1_test.csv")
city_count = data['城市'].nunique()
zone_count = data['区域'].nunique()
print(city_count)
print(zone_count)
data['建筑面积'] = pd.to_numeric(data['建筑面积'].str.replace('㎡', ''))
data_test['建筑面积'] = pd.to_numeric(data_test['建筑面积'].str.replace('㎡', ''))
data['均价']=data['价格']/data['建筑面积']
data['核心卖点'] = data['核心卖点'].notnull().astype(int)
data['周边配套'] = data['周边配套'].notnull().astype(int)
data['交通出行'] = data['交通出行'].notnull().astype(int)
data = data.drop(['抵押信息', 'lon', 'lat','小区名称','房屋户型','别墅类型','抵押信息','套内面积','上次交易'], axis=1)

7
90


In [81]:
data.describe()

,城市,区域,板块,价格,建筑面积,核心卖点,周边配套,交通出行,年份,均价
count,84133.000000,84133.000000,84133.000000,8.413300e+04,84133.000000,84133.000000,84133.000000,84133.000000,84133.000000,84133.000000
mean,2.894845,54.400794,434.420263,1.971953e+06,96.684309,0.805475,0.595557,0.614456,2021.070507,20294.731580
std,2.146294,27.870968,225.487506,2.639962e+06,63.328169,0.395837,0.490787,0.486727,0.882858,20372.957770
min,0.000000,2.000000,0.000000,7.828000e+04,10.000000,0.000000,0.000000,0.000000,2015.000000,65.074393
25%,1.000000,35.000000,244.000000,6.974000e+05,66.010000,1.000000,0.000000,0.000000,2021.000000,8636.471425
50%,2.000000,59.000000,435.000000,1.146500e+06,88.900000,1.000000,1.000000,1.000000,2021.000000,12163.084703
75%,5.000000,74.000000,628.000000,2.176000e+06,116.000000,1.000000,1.000000,1.000000,2022.000000,20662.535748
max,6.000000,102.000000,810.000000,7.995000e+07,10337.000000,1.000000,1.000000,1.000000,2022.000000,171775.429539


In [82]:
data_test.describe()

,ID,城市,区域,板块,建筑面积,抵押信息,lon,lat,年份
count,14786.000000,14786.000000,14786.000000,14786.000000,14786.000000,0.0,14786.000000,14786.000000,14786.000000
mean,7392.500000,2.679088,52.395712,426.121872,92.494791,NaN,115.443623,34.608035,2022.002232
std,4268.494875,1.945248,28.446181,227.521804,39.914068,NaN,5.853432,5.027996,0.047191
min,0.000000,0.000000,0.000000,0.000000,18.760000,NaN,106.197420,29.258156,2022.000000
25%,3696.250000,1.000000,29.000000,242.000000,66.780000,NaN,109.024538,30.601903,2022.000000
50%,7392.500000,3.000000,52.000000,427.000000,87.985000,NaN,116.313729,31.324525,2022.000000
75%,11088.750000,4.000000,76.000000,632.000000,109.980000,NaN,121.316778,39.873692,2022.000000
max,14785.000000,6.000000,102.000000,809.000000,594.380000,NaN,129.581642,46.340684,2023.000000


In [83]:
data_inner, data_outer = train_test_split(data, test_size=0.2, random_state=111)
#最小二乘法建模
X = data_inner[['建筑面积']]  # 自变量，需是二维数组形式
y = data_inner['价格']  # 因变量

# 添加常数项，因为OLS模型需要截距项
X = sm.add_constant(X) 
# 实例化OLS模型
model = sm.OLS(y, X)
# 拟合模型
results = model.fit()
# 打印模型摘要
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                     价格   R-squared:                       0.140
Model:                            OLS   Adj. R-squared:                  0.140
Method:                 Least Squares   F-statistic:                 1.096e+04
Date:                Thu, 03 Apr 2025   Prob (F-statistic):               0.00
Time:                        10:35:03   Log-Likelihood:            -1.0857e+06
No. Observations:               67306   AIC:                         2.171e+06
Df Residuals:                   67304   BIC:                         2.171e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.322e+05   1.67e+04     31.834      0.0

In [84]:
data_inner.describe()

,城市,区域,板块,价格,建筑面积,核心卖点,周边配套,交通出行,年份,均价
count,67306.000000,67306.000000,67306.000000,6.730600e+04,67306.000000,67306.000000,67306.000000,67306.000000,67306.000000,67306.000000
mean,2.891525,54.342258,434.461281,1.975985e+06,96.636222,0.805248,0.595504,0.614507,2021.069370,20336.607044
std,2.146416,27.826787,225.479235,2.642701e+06,66.188477,0.396013,0.490798,0.486715,0.882799,20406.935609
min,0.000000,2.000000,0.000000,7.828000e+04,10.000000,0.000000,0.000000,0.000000,2015.000000,65.074393
25%,1.000000,35.000000,244.000000,6.924000e+05,66.000000,1.000000,0.000000,0.000000,2021.000000,8644.566682
50%,2.000000,59.000000,434.000000,1.146500e+06,88.860000,1.000000,1.000000,1.000000,2021.000000,12168.669210
75%,5.000000,74.000000,627.000000,2.186000e+06,115.890000,1.000000,1.000000,1.000000,2022.000000,20807.778969
max,6.000000,102.000000,810.000000,7.995000e+07,10337.000000,1.000000,1.000000,1.000000,2022.000000,164860.197671


In [85]:
data_outer.describe()

,城市,区域,板块,价格,建筑面积,核心卖点,周边配套,交通出行,年份,均价
count,16827.000000,16827.000000,16827.000000,1.682700e+04,16827.000000,16827.000000,16827.000000,16827.000000,16827.000000,16827.000000
mean,2.908124,54.634932,434.256195,1.955825e+06,96.876650,0.806383,0.595769,0.614251,2021.075058,20127.234704
std,2.145819,28.046601,225.527213,2.628996e+06,50.287151,0.395144,0.490757,0.486786,0.883106,20236.216832
min,0.000000,2.000000,0.000000,7.970000e+04,10.000000,0.000000,0.000000,0.000000,2017.000000,1134.814815
25%,1.000000,35.000000,244.000000,7.076000e+05,66.545000,1.000000,0.000000,0.000000,2021.000000,8600.210912
50%,2.000000,59.000000,438.000000,1.151800e+06,89.000000,1.000000,1.000000,1.000000,2021.000000,12140.835103
75%,5.000000,78.000000,628.000000,2.152125e+06,116.800000,1.000000,1.000000,1.000000,2022.000000,20134.938286
max,6.000000,102.000000,810.000000,7.255000e+07,1200.000000,1.000000,1.000000,1.000000,2022.000000,171775.429539


In [86]:
from sklearn.model_selection import KFold

# 假设已经有处理好的 data_inner、data_outer 和 data_test
# data_inner = pd.read_csv('data_inner.csv')
# data_outer = pd.read_csv('data_outer.csv')

# 获取训练集预测值
y_train_pred = results.predict()
# 假设训练集真实值存储在y_train中（需要你根据实际情况确定变量名）
y_train = data_inner['价格']

# 计算训练集的MAE
mae_train = mean_absolute_error(y_train, y_train_pred)
# 计算训练集的RMSE
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))

print(f"训练集平均绝对误差（MAE）: {mae_train}")
print(f"训练集均方根误差（RMSE）: {rmse_train}")

# 从data_test中提取自变量（建筑面积）并添加常数项（因为OLS模型有截距项）
X_test = data_outer['建筑面积']
X_test = sm.add_constant(X_test)

# 对样本外数据进行预测
y_test_pred = results.predict(X_test)
# 样本外数据的真实值
y_test = data_outer['价格']

# 计算样本外数据的MAE
mae_test = mean_absolute_error(y_test, y_test_pred)
# 计算样本外数据的RMSE
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"样本外数据平均绝对误差（MAE）: {mae_test}")
print(f"样本外数据均方根误差（RMSE）: {rmse_test}")

# 进行6重交叉验证
kf = KFold(n_splits=6, shuffle=True, random_state=42)
mae_scores = []
rmse_scores = []

# 提取训练集的自变量并添加常数项
X_train = data_inner['建筑面积']
X_train = sm.add_constant(X_train)

for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # 拟合模型
    model = sm.OLS(y_train_fold, X_train_fold).fit()

    # 预测验证集
    y_val_pred = model.predict(X_val_fold)

    # 计算MAE和RMSE
    mae = mean_absolute_error(y_val_fold, y_val_pred)
    rmse = np.sqrt(mean_squared_error(y_val_fold, y_val_pred))

    mae_scores.append(mae)
    rmse_scores.append(rmse)

# 计算交叉验证的平均MAE和RMSE
cv_mae_mean = np.mean(mae_scores)
cv_rmse_mean = np.mean(rmse_scores)

print(f"6重交叉验证平均绝对误差（MAE）均值: {cv_mae_mean}")
print(f"6重交叉验证均方根误差（RMSE）均值: {cv_rmse_mean}")

训练集平均绝对误差（MAE）: 1391419.0358043811
训练集均方根误差（RMSE）: 2450680.725105904
样本外数据平均绝对误差（MAE）: 1373196.755244483
样本外数据均方根误差（RMSE）: 2338163.8098671995
6重交叉验证平均绝对误差（MAE）均值: 1393172.111089269
6重交叉验证均方根误差（RMSE）均值: 2529940.3961120406


In [92]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import os

# 假设已经有处理好的 data_inner、data_outer 和 data_test
# data_inner = pd.read_csv('data_inner.csv')
# data_outer = pd.read_csv('data_outer.csv')
# data_test = pd.read_csv('data_test.csv')

# 选择要使用的特征
selected_features = ['城市', '区域', '板块', '环线']

# 对 data_inner 进行处理
target_inner = data_inner['均价']
features_inner = data_inner[selected_features]

# 对分类变量进行独热编码
encoder_inner = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
encoded_data_inner = encoder_inner.fit_transform(features_inner)
features_processed_inner = pd.DataFrame.sparse.from_spmatrix(
    encoded_data_inner, columns=encoder_inner.get_feature_names_out(selected_features)
)

# 对 data_outer 进行相同处理
target_outer = data_outer['均价']
features_outer = data_outer[selected_features]

encoded_data_outer = encoder_inner.transform(features_outer)
features_processed_outer = pd.DataFrame.sparse.from_spmatrix(
    encoded_data_outer, columns=encoder_inner.get_feature_names_out(selected_features)
)

# 定义 Lasso 模型
alpha = 1.0  # 可根据实际情况调整
lasso = Lasso(alpha=alpha)

# 在样本内（data_inner）上训练模型
lasso.fit(features_processed_inner, target_inner)

# 在样本内进行预测
y_pred_inner = lasso.predict(features_processed_inner)

# 计算样本内的 MAE 和 RMSE
mae_inner = mean_absolute_error(target_inner, y_pred_inner)
rmse_inner = np.sqrt(mean_squared_error(target_inner, y_pred_inner))

print(f"样本内平均绝对误差（MAE）: {mae_inner}")
print(f"样本内均方根误差（RMSE）: {rmse_inner}")

# 在样本外（data_outer）进行预测
y_pred_outer = lasso.predict(features_processed_outer)

# 计算样本外的 MAE 和 RMSE
mae_outer = mean_absolute_error(target_outer, y_pred_outer)
rmse_outer = np.sqrt(mean_squared_error(target_outer, y_pred_outer))

print(f"样本外平均绝对误差（MAE）: {mae_outer}")
print(f"样本外均方根误差（RMSE）: {rmse_outer}")

# 对 data_test 进行相同处理
features_test = data_test[selected_features]
encoded_data_test = encoder_inner.transform(features_test)
features_processed_test = pd.DataFrame.sparse.from_spmatrix(
    encoded_data_test, columns=encoder_inner.get_feature_names_out(selected_features)
)

#使用训练好的模型对 data_test 进行预测
y_pred_test = lasso.predict(features_processed_test)

# 将预测结果添加到 data_test 数据框中
data_test['预测均价'] = y_pred_test

# 将“预测均价”列转换为数值类型，无法转换的值设为NaN
data_test['预测均价'] = pd.to_numeric(data_test['预测均价'], errors='coerce')

# 定义保存路径
save_path = "C:\\桌面文件下载在这里\\新建文件夹"
# 如果路径不存在，创建路径
if not os.path.exists(save_path):
    os.makedirs(save_path)

# 保存 data_test 为 CSV 文件
data_test.to_csv(os.path.join(save_path, 'data_test_with_prediction1.csv'), index=False)

print(f"已将 data_test 保存到 {save_path} 路径下的 data_test_with_prediction.csv 文件中。")

样本内平均绝对误差（MAE）: 3091.2947884277423
样本内均方根误差（RMSE）: 5623.312479560482
样本外平均绝对误差（MAE）: 3109.5287861033717
样本外均方根误差（RMSE）: 5783.517467215895
已将 data_test 保存到 C:\桌面文件下载在这里\新建文件夹 路径下的 data_test_with_prediction.csv 文件中。


In [91]:
# 选择要使用的特征
selected_features = ['城市', '区域', '板块', '环线']

# 对 data_inner 进行处理
target_inner = data_inner['均价']
features_inner = data_inner[selected_features]

# 对分类变量进行独热编码
encoder_inner = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
encoded_data_inner = encoder_inner.fit_transform(features_inner)
features_processed_inner = pd.DataFrame.sparse.from_spmatrix(
    encoded_data_inner, columns=encoder_inner.get_feature_names_out(selected_features)
)

# 对 data_outer 进行相同处理
target_outer = data_outer['均价']
features_outer = data_outer[selected_features]

encoded_data_outer = encoder_inner.transform(features_outer)
features_processed_outer = pd.DataFrame.sparse.from_spmatrix(
    encoded_data_outer, columns=encoder_inner.get_feature_names_out(selected_features)
)

# 对 data_test 进行相同处理
features_test = data_test[selected_features]
encoded_data_test = encoder_inner.transform(features_test)
features_processed_test = pd.DataFrame.sparse.from_spmatrix(
    encoded_data_test, columns=encoder_inner.get_feature_names_out(selected_features)
)

# 定义 Ridge 模型
alpha = 0.1  # 可根据实际情况调整
ridge = Ridge(alpha=alpha)

# 在样本内（data_inner）上训练模型
ridge.fit(features_processed_inner, target_inner)

# 在样本内进行预测
y_pred_inner = ridge.predict(features_processed_inner)

# 计算样本内的 MAE 和 RMSE
mae_inner = mean_absolute_error(target_inner, y_pred_inner)
rmse_inner = np.sqrt(mean_squared_error(target_inner, y_pred_inner))

print(f"样本内平均绝对误差（MAE）: {mae_inner}")
print(f"样本内均方根误差（RMSE）: {rmse_inner}")

# 在样本外（data_outer）进行预测
y_pred_outer = ridge.predict(features_processed_outer)

# 计算样本外的 MAE 和 RMSE
mae_outer = mean_absolute_error(target_outer, y_pred_outer)
rmse_outer = np.sqrt(mean_squared_error(target_outer, y_pred_outer))

print(f"样本外平均绝对误差（MAE）: {mae_outer}")
print(f"样本外均方根误差（RMSE）: {rmse_outer}")

# 使用训练好的模型对 data_test 进行预测
y_pred_test = ridge.predict(features_processed_test)

# 确保预测结果是数字形式
y_pred_test = np.array(y_pred_test, dtype=float)

# 将预测结果添加到 data_test 数据框中
data_test['预测均价'] = y_pred_test

# 定义保存路径
save_path = "C:\\桌面文件下载在这里\\新建文件夹"
# 如果路径不存在，创建路径
if not os.path.exists(save_path):
    os.makedirs(save_path)

# 保存 data_test 为 CSV 文件
data_test['预测均价'] = pd.to_numeric(data_test['预测均价'], errors='coerce')  # 将列转换为数值类型，无法转换的值设为NaN
data_test.to_csv(os.path.join(save_path, 'data_test_with_prediction2.csv'), index=False, float_format='%.2f')  # 保存时指定浮点数格式，这里保留两位小数

print(f"已将 data_test 保存到 {save_path} 路径下的 data_test_with_prediction.csv 文件中。")


# 6 重交叉验证
#cv_scores_mae = -cross_val_score(ridge, features_processed_inner, target_inner, cv=6, scoring='neg_mean_absolute_error')
#cv_scores_rmse = np.sqrt(-cross_val_score(ridge, features_processed_inner, target_inner, cv=6, scoring='neg_mean_squared_error'))

#print(f"6 重交叉验证平均绝对误差（MAE）均值: {cv_scores_mae.mean()}")
#print(f"6 重交叉验证均方根误差（RMSE）均值: {cv_scores_rmse.mean()}")
    
    

样本内平均绝对误差（MAE）: 2942.8250396215626
样本内均方根误差（RMSE）: 5459.316012104374
样本外平均绝对误差（MAE）: 2980.7207068213716
样本外均方根误差（RMSE）: 5615.019710368942
已将 data_test 保存到 C:\桌面文件下载在这里\新建文件夹 路径下的 data_test_with_prediction.csv 文件中。
